In [1]:
# n = 104729 * 103591
# step = 25000
# B = 25000

In [2]:
n = 97 * 31
B = 30
step = 50

In [3]:
import decimal
import numpy as np
from time import time

## lib.py

In [4]:
def GCD(m,n):
    mult = 1
    if m > n:
        m = m % n
    elif n > m:
        n = n % m

    while True:
        if m == 0 or n == 0 or m == n:
            return mult*max(n,m)
        if m == 1 or n == 1:
            return mult

        mm2 = m % 2
        nm2 = n % 2
        if mm2 == 0 and nm2 == 0:
            mult *= 2
            m = m//2
            n = n//2
        elif mm2 == 0 and nm2 != 0:
            m = m//2
        elif mm2 != 0 and nm2 == 0:
            n = n//2
        elif mm2 != 0 and nm2 != 0:
            if n > m:
                piv = (n-m)//2
                n = m
                m = piv
            elif n < m:
                m = (m-n)//2

In [5]:
def eratosthenes(n):
    numbers = list(range(2, n + 1))
    for number in numbers:
        if number != 0:
            for candidate in range(2 * number, n+1, number):
                numbers[candidate-2] = 0
    return list(filter(lambda x: x != 0, numbers))

In [6]:
class Q:
    def __init__(self,n):
        self.n = n
        self.m = int(decimal.Decimal(n).sqrt() + 1)
    def __call__(self,x):
        return (x + self.m)**2 - self.n

In [7]:
def legendre(a, p):
    return pow(a, (p - 1) // 2, p)

In [8]:
def tonelli(n, p):
    assert legendre(n, p) == 1, "not a square (mod p)"
    q = p - 1
    s = 0
    while q % 2 == 0:
        q //= 2
        s += 1
    if s == 1:
        return pow(n, (p + 1) // 4, p)

    z = 2
    for z in range(2, p):
        if p - 1 == legendre(z, p):
            break
    c = pow(z, q, p)
    r = pow(n, (q + 1) // 2, p)
    t = pow(n, q, p)
    m = s
    t2 = 0
    while (t - 1) % p != 0:
        t2 = (t * t) % p
        for i in range(1, m):
            if (t2 - 1) % p == 0:
                break
            t2 = (t2 * t2) % p
        b = pow(c, 1 << (m - i - 1), p)
        r = (r * b) % p
        c = (b * b) % p
        t = (t * c) % p
        m = i
    return r

In [9]:
def smooth_region(L1, L2, q, primes):
    res0 = list(range(L1, L2))
    res1 = np.array([q(x) for x in range(L1, L2)])
    res2 = np.zeros((len(res0), len(primes)), dtype="int8")

    s = [[] for _ in range(len(primes.r))]
    for smooth_idx, prime in enumerate(primes):
        for r in primes.r[smooth_idx]:
            k = L1 // prime
            while r + k*prime >= L1:
                k -= 1
            k+=1
            s[smooth_idx].append(r + k*prime)

    for prime_idx, prime in enumerate(primes):
        for s_i in s[prime_idx]:
            if s_i < L2:
                # гарантируем что начиная с s_1 - L1, каждое число через prime
                # делится на этот prime хотя бы 1 раз
                res2[s_i - L1::prime, prime_idx] += 1
                res1[s_i - L1::prime] //= prime
                # дальнешее деление проверяется вручную
                for smooth_idx in range(s_i, L2, prime):
                    x = smooth_idx - L1
                    while res1[x] % prime == 0:
                        res1[x] //= prime
                        res2[x, prime_idx] += 1

    ans = []
    for smooth_idx, qx in enumerate(res1):
        if abs(qx) == 1:
            ans.append([res0[smooth_idx],q(res0[smooth_idx]),np.copy(res2[smooth_idx])])
            
    return ans

## Primes

In [10]:
class Primes:
    def __init__(self, n, B, q):
        primes = eratosthenes(B)
        self.p = []
        self.r = []
        q = Q(n)

        if legendre(n % primes[0], primes[0]) == 1:
            tr = tonelli(n,primes[0])
            r = [(tr - q.m) % primes[0]]
            self.p.append(primes[0])
            self.r.append(r)

        # Для каждого прайма проверяем Лежандра и находим корни Тонелли Шенксом
        for i in range(1,len(primes)):
            if legendre(n % primes[i], primes[i]) == 1:
                tr = tonelli(n,primes[i])
                r = [(tr - q.m) % primes[i],(primes[i] - tr - q.m) % primes[i]]
                self.p.append(primes[i])
                self.r.append(r)

        print("primes len",len(self.p))
    def __getitem__(self,i):
        return self.p[i]
    def __len__(self):
        return len(self.p)

## colors

In [11]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

def color(data,col):
    """
    data: object which will be colored
    col:  string representing color. possible values are:
        ['data', '%', 'time', 'strong']
    """
    if col == "data":
        # pink
        return bcolors.HEADER+str(data)+bcolors.ENDC
    elif col == "%":
        # green (leave two digits after dot)
        return bcolors.OKGREEN+"{:.02f}".format(float(data))+bcolors.ENDC+" %"
    elif col == "time":
        # light blue
        return bcolors.OKCYAN+str(data)+bcolors.ENDC+" sec"
    elif col == "strong":
        # red
        return bcolors.FAIL+str(data)+bcolors.ENDC

## Sieve

In [12]:
def suive(q, primes):
    print("step:",step)
    k = 1
    # found_smooth = 0
    smooth_numbers = []
    while q((k-1)*step) < n:
        ans = smooth_region((k-1)*step,k*step,q,primes)
        for i in range(len(ans)):
            smooth_numbers.append(ans[i])
        ans = smooth_region(-k*step,-(k-1)*step,q,primes)
        for i in range(len(ans)):
            smooth_numbers.append(ans[i])
        k+=1
        print("Total number of smooth numbers:",len(smooth_numbers))
        if len(smooth_numbers) > len(primes):
            # Выброс из функции
            return smooth_numbers

## Matrix solver

In [13]:
class Matrix_solver:
    def __init__(self, primes):
        self.matrix = []
        self.primes = primes
        self.gaus = []
        self.num_smooth_numbers = 0
        self.matrix = np.zeros((len(primes), len(primes)), dtype='uint8')

    def add(self, smooth_number):
        if(self.num_smooth_numbers == len(self.primes)):
            return False
        self.matrix[:,self.num_smooth_numbers] = smooth_number
        self.num_smooth_numbers += 1

    def solve(self,smooth_numbers):
        # c++ matrix solve

        ans = [None,None]
        lineal_rows = gaus_solve(self.matrix)

        for zero_column in range(len(lineal_rows[0])):
            b = [] #indexes of smooth numbers i guess
            for i in range(len(lineal_rows)):
                if lineal_rows[i][zero_column]:
                    b.append(i)
            # got vector b as indexes of posible ans

            left = 1
            right = []
            for i in b:
                left *= int(decimal.Decimal(n).sqrt() + 1) + smooth_numbers[i][0]
                right.append(smooth_numbers[i][2])
            true_right = int(1)

            right_piv = np.zeros(len(self.primes), int)

            print(len(right), right[0][0])
            for r in right:
                right_piv += r
            
            for j in range(len(right_piv)):
                right_piv[j] //= 2

            for j in range(len(right_piv)):
                true_right *= pow(self.primes[j], int(right_piv[j]))

            gcd = min(GCD(abs(int(left+true_right)), n), GCD(abs(int(left-true_right)), n))
            if gcd > 1 and (n // gcd) * gcd == n:
                print(color("Solve Done",'strong'))
                ans = [gcd, n//gcd]
                break
            else:
                print("guess was",color('wrong', 'strong'),"keep search!\n")
        return ans

## Factor

In [14]:
def Factor(n,B):
    pass

## index

In [15]:
q = Q(n)
primes = Primes(n,B,q)
len(primes)
smooth_numbers = suive(q,primes)
len(smooth_numbers)

primes len 8
step: 50
Total number of smooth numbers: 31


31

In [16]:
matrix = Matrix_solver(primes.p)

In [17]:
for smooth in smooth_numbers:
    matrix.add(smooth[2])

In [18]:
matrix.matrix

array([[1, 1, 0, 1, 0, 1, 1, 0],
       [2, 0, 1, 1, 2, 1, 1, 1],
       [0, 0, 1, 1, 0, 1, 0, 2],
       [0, 2, 0, 0, 2, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1, 0, 0]], dtype=uint8)

In [19]:
# # possibe outcome [None,None] or if we LUCKY give ans as [gcd,n//gcd]
# solve = matrix.solve(smooth_numbers)

In [20]:
print(primes.p)

[2, 3, 7, 11, 13, 17, 19, 29]


In [21]:
print(primes.r)

[[0], [0, 1], [3, 6], [9, 2], [8, 12], [3, 6], [11, 12], [10, 25]]


In [22]:
smooth_numbers

[[0, 18, array([1, 2, 0, 0, 0, 0, 0, 0], dtype=int8)],
 [2, 242, array([1, 0, 0, 2, 0, 0, 0, 0], dtype=int8)],
 [3, 357, array([0, 1, 1, 0, 0, 1, 0, 0], dtype=int8)],
 [6, 714, array([1, 1, 1, 0, 0, 1, 0, 0], dtype=int8)],
 [9, 1089, array([0, 2, 0, 2, 0, 0, 0, 0], dtype=int8)],
 [10, 1218, array([1, 1, 1, 0, 0, 0, 0, 1], dtype=int8)],
 [12, 1482, array([1, 1, 0, 0, 1, 0, 1, 0], dtype=int8)],
 [13, 1617, array([0, 1, 2, 1, 0, 0, 0, 0], dtype=int8)],
 [20, 2618, array([1, 0, 1, 1, 0, 1, 0, 0], dtype=int8)],
 [24, 3234, array([1, 1, 2, 1, 0, 0, 0, 0], dtype=int8)],
 [25, 3393, array([0, 2, 0, 0, 1, 0, 0, 1], dtype=int8)],
 [31, 4389, array([0, 1, 1, 1, 0, 0, 1, 0], dtype=int8)],
 [34, 4914, array([1, 3, 1, 0, 1, 0, 0, 0], dtype=int8)],
 [-48, -2958, array([1, 1, 0, 0, 0, 1, 0, 1], dtype=int8)],
 [-46, -2926, array([1, 0, 1, 1, 0, 0, 1, 0], dtype=int8)],
 [-45, -2907, array([0, 2, 0, 0, 0, 1, 1, 0], dtype=int8)],
 [-36, -2646, array([1, 3, 2, 0, 0, 0, 0, 0], dtype=int8)],
 [-33, -2523, ar